## Fine-Tuning GPT-2 Model


### Import:

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('drive/MyDrive/cmpt413_proj/')

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn.functional as F
import pandas as pd
from transformers import pipeline

For some reason when you install accelerate v0.25.0, you need to restart the session for changes to take place

In [ ]:
#@title Change .csv to .txt for dataset
import csv

# Replace 'your_file.csv' with the path to your CSV file
csv_file_path = 'formatted_data_tagged_test.csv'

# Replace 'output.txt' with the desired output TXT file path
txt_file_path = 'formatted_data_tagged_test.txt'

# Read the CSV and write its content to a TXT file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)

    with open(txt_file_path, 'w') as txt_file:
        for row in csv_reader:
            txt_file.write(' '.join(row) + '\n')


### Data Pre-Processing:

Performs any necessary preprocessing of the dataset.

In [ ]:
import pandas as pd

dfs = []
file_paths =  ['data/quest_dataset_up_to_kalimdor.csv', 'data/quest_dataset_kalimdor_to_profession.csv','data/quest_dataset_profession_to_end.csv', 'data/quest_dataset_wotlk.csv', 'data/quest_dataset_classic.csv']
for file_path in file_paths:
  df = pd.read_csv(file_path)
  df.columns = ["id", "name", "objective", "description"]
  dfs.append(df)
combined_df = pd.concat(dfs, ignore_index=True)
df = combined_df

df.columns = ["id", "name", "objective", "description"]
df

In [ ]:
import pandas as pd

# Drop the ID column and any other unwanted columns
df = df.drop(columns=['id'])

df = df.drop_duplicates(keep='last')

# Reorder the indices of the DataFrame
df.reset_index(drop=True, inplace=True)

# Transform the dataset according to the specified format
def format_quest(row):
    title = f"<|startoftext|> {row['name']}"
    objective = f"<|obj|> {row['objective']}"
    description = f"<|text|> {row['description']} <|endoftext|>"
    return f"{title} {objective} {description}"

df['formatted_quest'] = df.apply(format_quest, axis=1)

df['formatted_quest']

In [ ]:
# Save the transformed dataset to a new CSV file
output_file_path_train = 'formatted_data_tagged_train.csv'
output_file_path_test = 'formatted_data_tagged_test.csv'

from sklearn.model_selection import train_test_split

df2 = df['formatted_quest']
train, test = train_test_split(df2, test_size=0.1)

train.to_csv(output_file_path_train, index=False, header=True)
test.to_csv(output_file_path_test, index=False, header=True)

output_file_path_train
output_file_path_test

### Load Model

Load the GPT-2 model and tokenizer.

In [ ]:
from transformers import AutoModelWithLMHead
from transformers import AutoTokenizer

MODEL_NAME = 'gpt2'
model = AutoModelWithLMHead.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### Freeze layers (optional):

Optionally freeze the first layers that are responsible for general knowledge, as well as the first header layers.

In [ ]:
N = 4  # Number of layers to freeze
for i, (name, param) in enumerate(model.named_parameters()):
    if i < N:
        param.requires_grad = False

for i, (name, param) in enumerate(model.named_parameters()):
    if 'transformer.h' in name:
        if 'transformer.h.0' in name:
          param.requires_grad = False
        else:
          param.requires_grad = True

# Check if the first N layers are frozen
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer {name} is trainable.")
    else:
        print(f"Layer {name} is frozen.")

Layer transformer.wte.weight is frozen.
Layer transformer.wpe.weight is frozen.
Layer transformer.h.0.ln_1.weight is frozen.
Layer transformer.h.0.ln_1.bias is frozen.
Layer transformer.h.0.attn.c_attn.weight is frozen.
Layer transformer.h.0.attn.c_attn.bias is frozen.
Layer transformer.h.0.attn.c_proj.weight is frozen.
Layer transformer.h.0.attn.c_proj.bias is frozen.
Layer transformer.h.0.ln_2.weight is frozen.
Layer transformer.h.0.ln_2.bias is frozen.
Layer transformer.h.0.mlp.c_fc.weight is frozen.
Layer transformer.h.0.mlp.c_fc.bias is frozen.
Layer transformer.h.0.mlp.c_proj.weight is frozen.
Layer transformer.h.0.mlp.c_proj.bias is frozen.
Layer transformer.h.1.ln_1.weight is trainable.
Layer transformer.h.1.ln_1.bias is trainable.
Layer transformer.h.1.attn.c_attn.weight is trainable.
Layer transformer.h.1.attn.c_attn.bias is trainable.
Layer transformer.h.1.attn.c_proj.weight is trainable.
Layer transformer.h.1.attn.c_proj.bias is trainable.
Layer transformer.h.1.ln_2.weight 

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
model = model.to('cuda:0')

### Load Dataset

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import TextDataset

txt_file_path = 'formatted_data_tagged_train.txt'
txt_file_path = 'data/GPT2_WOWHead_dataset.txt'
txt_file_path_test = 'formatted_data_tagged_test.csv'

train_data = TextDataset(
    tokenizer=tokenizer,
    file_path=str(txt_file_path),
    block_size=128,
)
test_data = TextDataset(
    tokenizer=tokenizer,
    file_path=str(txt_file_path_test),
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (2989064 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
train_data

In [ ]:
from pathlib import Path
import json

### Train:

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments

model_path = Path('./gpt2-quests')
logs_path = Path('./logs_quests')

training_args = TrainingArguments(
  output_dir=str(model_path),     # output directory
  logging_dir=str(logs_path),     # logs directory
  overwrite_output_dir=True,      # overwrite the content of the output directory
  num_train_epochs=4,            # number of training epochs
  per_device_train_batch_size=4, # batch size for training
  per_device_eval_batch_size=4,  # batch size for evaluation
  eval_steps=2000,                 # number of update steps between two evaluations
  save_steps=10000,                 # after how many steps model is saved
  warmup_steps=500,               # number of warmup steps for learning rate scheduler
  learning_rate=3e-5,
)

trainer = Trainer(
  model=model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=train_data,
  eval_dataset=test_data,
#   prediction_loss_only=True,
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.550300
1000,3.146400
1500,3.073200
2000,3.016300
2500,2.980100
3000,2.971500
3500,2.938600
4000,2.926200
4500,2.900100
5000,2.898200


TrainOutput(global_step=20040, training_loss=2.7823051094770905, metrics={'train_runtime': 2904.6747, 'train_samples_per_second': 27.594, 'train_steps_per_second': 6.899, 'total_flos': 5235769737216000.0, 'train_loss': 2.7823051094770905, 'epoch': 4.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 2.8743784427642822,
 'eval_runtime': 30.1692,
 'eval_samples_per_second': 75.043,
 'eval_steps_per_second': 18.761,
 'epoch': 2.0}

In [ ]:
trainer.save_model('model_10_new')

### Evaluate:

In [ ]:
from transformers import pipeline

def generate(title, temperature, top_k):
  generator = pipeline('text-generation', model='data/ft_gpt2_run10', tokenizer=tokenizer)
  return generator(
        f'{title}',
        max_length=128,
        temperature=temperature,
        top_k=top_k
  )[0]['generated_text']

In [ ]:
print(generate('<|startoftext|> Build a Rocket Ship <|obj|> Find raw materials around this exo-planet to build a rocket ship! ', temperature=0.8, top_k=60))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<|startoftext|> Build a Rocket Ship <|obj|> Find raw materials around this exo-planet to build a rocket ship!  <|text|> <The blueprint explains how to build an exo-planet, but there is only one piece. ><name>, you'll need to find the raw materials around this exo-planet and assemble a rocket ship. If you're a rocket scientist, you'll also want to know how to build a rocket ship! The blueprint says that the rocket ship should be built using materials from the planet's surface, but there is only one piece. > 


In [ ]:
def split_string_at_separator(input_string, separator):
    # Find the index of the separator
    index = input_string.find(separator)

    # Check if separator is found
    if index != -1:
        # Include the separator in the first part
        first_part = input_string[:index + len(separator)]
        second_part = input_string[index + len(separator):]
    else:
        # If separator not found, return the whole string as the first part
        first_part = input_string
        second_part = ""

    return first_part, second_part

In [ ]:
# Check if GPU is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'

generator = pipeline('text-generation', model='data/ft_gpt2_run10', tokenizer=tokenizer)

dataset_test_path = 'formatted_data_tagged_test.csv'
df_test = pd.read_csv(dataset_test_path)
df_test.columns = ['sentence']

references = []
outputs = []

max_step = 500

for k, sentence in enumerate(df_test['sentence']):
    input, output = split_string_at_separator(sentence, '<|text|>')
    model_output = generator(
                            f'{input}',
                            max_length=128,
                            temperature=0.8,
                            top_k=60
                      )[0]['generated_text']
    references.append(sentence)
    outputs.append(model_output)

    print("input: {}".format(input))
    print(model_output)

    if k >= max_step:
      break

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: <|startoftext|> The Pools of Youth <|obj|> Obtain some of the Water of Youth.  <|text|>
<|startoftext|> The Pools of Youth <|obj|> Obtain some of the Water of Youth.  <|text|> You must know, <name>, that the people of this forest have a thirst for water, and that they thirst for power.  There are creatures from the swamp with a thirst for magic, of course, but most are very powerful.  Their magic has a way of harnessing the power of the water itself, but it is an unstable mixture...  I know of an old man who once drank the waters of youth.  He named it the pools of youth


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: <|startoftext|> Highland Fishing <|obj|> Greetings, friend! I see you're getting to know the ways of tuskarr, and thus the ways of the fish.  <|text|>
<|startoftext|> Highland Fishing <|obj|> Greetings, friend! I see you're getting to know the ways of tuskarr, and thus the ways of the fish.  <|text|> I see you're getting to know the ways of tuskarr, and thus the ways of the fish.  I think you're ready for a challenge, as my father taught you. The trout are a big part of the Highland fishing tradition. While you're out, I'd like you to fish one of the trout from the Hibernal River.  That's where you'll


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: <|startoftext|> Flexing for Nougat <|obj|> Do a flex emote at Innkeeper Gryshka, and in exchange you'll receive Orgrimmar Nougat! A Hallow's End quest.  <|text|>
<|startoftext|> Flexing for Nougat <|obj|> Do a flex emote at Innkeeper Gryshka, and in exchange you'll receive Orgrimmar Nougat! A Hallow's End quest.  <|text|> I may have overdone it, but I don't want you to go overboard.  I've been doing a little bit of math over in the inn for the past couple of months, and I have a feeling we will be getting a little better at our own sewing and fabrication.  I can do some fancy work if you lend me a hand. Enter the


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


input: <|startoftext|> Craftsman's Writ - Plated Armorfish <|obj|> Deliver 30 Plated Armorfish and the Craftsman's Writ - Plated Armorfish - to Packmaster Stonebruiser at Light's Hope Chapel in the Eastern Plaguelands.  <|text|>
<|startoftext|> Craftsman's Writ - Plated Armorfish <|obj|> Deliver 30 Plated Armorfish and the Craftsman's Writ - Plated Armorfish - to Packmaster Stonebruiser at Light's Hope Chapel in the Eastern Plaguelands.  <|text|> Welcome to Light's Hope Chapel, <class>.  What a beautiful day it has been.  Now that we have the armorfish for our recipes, the rest of us will have to work hard to make them.  Our next job is to make the armorfish.  I've got orders for


KeyboardInterrupt: ignored

In [ ]:
#@title Export

import csv

if not os.path.exists('output'):
    os.makedirs('output')

csv_file_path = 'output/formatted_dataset_tagged_test_out.csv'

# Write the list to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Writing each string as a new row
    for string in outputs:
        csv_writer.writerow([string])

print(f"List of strings written to {csv_file_path}")

List of strings written to formatted_dataset_tagged_test_out.csv


### Compute BLEU Score

Compute BLEU score between Fine-Tuned output and input reference texts to ensure structural integrity is maintained. This was justified to ensure that the output of this fine-tuned network adheres to the format of the dataset, which is important as it sometimes requires it to generate special tokens.

In [ ]:
# Load in references and preds from test file
# Check if GPU is available and set the device accordingly

import pandas as pd

dataset_test_path = 'formatted_data_tagged_test.csv'
dataset_test_path_out = 'output/formatted_dataset_tagged_test_out.csv'

df_test = pd.read_csv(dataset_test_path)
df_test.columns = ['sentence']

df_test_out = pd.read_csv(dataset_test_path_out, header=None, names=['sentence'])

references = []
outputs = []

max_step = 500

for k, sentence in enumerate(df_test['sentence']):
    references.append(sentence)
    outputs.append(df_test_out['sentence'][k])

    if k >= max_step or k >= len(df_test_out['sentence']):
      break

In [ ]:
import sys, re
import sacrebleu

bleu = sacrebleu.metrics.BLEU(effective_order=True)

def compute_bleu(references, output_data):
    bleu_score = 0.0
    if len(references) == len(output_data):
        score = 0.0
        total = 0.0
        for i, line in enumerate(output_data):
            print("reference: {}".format(references[i]))
            print("generated: {}".format(line))
            r = references[i]
            ref = [r]
            h = line
            score += bleu.sentence_score(h, ref).score
            total += 1.
        bleu_score = score / total
    return bleu_score

In [ ]:
print(compute_bleu(references, outputs))

reference: <|startoftext|> The Pools of Youth <|obj|> Obtain some of the Water of Youth.  <|text|> North! Go north to the Pools of Youth. Get some of the water but be on your guard. The mogu list this as a "high value" target. Kang Bramblestaff will meet you there if he has not already arrived by your side.  <|endoftext|>
generated: <|startoftext|> The Pools of Youth <|obj|> Obtain some of the Water of Youth.  <|text|> You must know, <name>, that the people of this forest have a thirst for water, and that they thirst for power.  There are creatures from the swamp with a thirst for magic, of course, but most are very powerful.  Their magic has a way of harnessing the power of the water itself, but it is an unstable mixture...  I know of an old man who once drank the waters of youth.  He named it the pools of youth
reference: <|startoftext|> Highland Fishing <|obj|> Greetings, friend! I see you're getting to know the ways of tuskarr, and thus the ways of the fish.  <|text|> Greetings, fr